**Reconstruction LSTM Autoencoder**

In [1]:
# lstm autoencoder recreate sequence
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

# define input sequence
sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)
plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')

# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

Using TensorFlow backend.
W0814 01:25:51.437162 139998407567232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 01:25:51.476914 139998407567232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 01:25:51.484309 139998407567232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 01:25:52.119828 139998407567232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0814 01:25:52.433589 13999840

[0.10484007 0.20024277 0.29893196 0.39903706 0.49956563 0.5999912
 0.7001451  0.80012715 0.900236  ]


**Prediction LSTM Autoencoder**

In [2]:
# define input sequence
seq_in = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))

# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
plot_model(model, show_shapes=True, to_file='predict_lstm_autoencoder.png')

# fit model
model.fit(seq_in, seq_out, epochs=300, verbose=0)

# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat[0,:,0])

[0.16613358 0.28874403 0.40254888 0.5093199  0.6104444  0.7070395
 0.80003124 0.89021   ]


In [0]:
#  lstm autoencoder reconstruct and predict sequence
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

**Composite LSTM Autoencoder**

In [5]:
# define input sequence
seq_in = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))

# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

# define encoder
visible = Input(shape=(n_in,1))
encoder = LSTM(100, activation='relu')(visible)

# define reconstruct decoder
decoder1 = RepeatVector(n_in)(encoder)
decoder1 = LSTM(100, activation='relu', return_sequences=True)(decoder1)
decoder1 = TimeDistributed(Dense(1))(decoder1)

# define predict decoder
decoder2 = RepeatVector(n_out)(encoder)
decoder2 = LSTM(100, activation='relu', return_sequences=True)(decoder2)
decoder2 = TimeDistributed(Dense(1))(decoder2)

# tie it together
model = Model(inputs=visible, outputs=[decoder1, decoder2])
model.compile(optimizer='adam', loss='mse')
plot_model(model, show_shapes=True, to_file='composite_lstm_autoencoder.png')

# fit model
model.fit(seq_in, [seq_in,seq_out], epochs=300, verbose=0)

# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat)

[array([[[0.11021672],
        [0.20789155],
        [0.3045485 ],
        [0.40044165],
        [0.49661556],
        [0.5941415 ],
        [0.6941657 ],
        [0.7979694 ],
        [0.90703845]]], dtype=float32), array([[[0.16879326],
        [0.29085186],
        [0.4024964 ],
        [0.5075912 ],
        [0.60796845],
        [0.7050089 ],
        [0.79981226],
        [0.8933235 ]]], dtype=float32)]


**Keep Standalone LSTM Encoder**

In [6]:
# define input sequence
sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)
# connect the encoder LSTM as the output layer
model = Model(inputs=model.inputs, outputs=model.layers[0].output)
plot_model(model, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
yhat = model.predict(sequence)
print(yhat.shape)
print(yhat)

(1, 100)
[[0.00000000e+00 8.40858147e-02 1.02115028e-01 0.00000000e+00
  4.29800265e-02 2.34062616e-02 2.81083435e-02 5.50449751e-02
  6.30786642e-02 0.00000000e+00 6.31890222e-02 3.07526104e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.03702806e-02 9.41602662e-02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 8.45623389e-02 9.90664214e-02
  7.69042224e-02 0.00000000e+00 1.33752739e-02 8.61792490e-02
  1.01291709e-01 0.00000000e+00 0.00000000e+00 8.61476734e-02
  0.00000000e+00 2.44012428e-03 0.00000000e+00 0.00000000e+00
  0.00000000e+00 8.46719816e-02 5.65234944e-03 7.17138946e-02
  6.84074983e-02 1.07211845e-05 3.32521312e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 6.15448505e-02 6.77432269e-02
  1.12942457e-01 2.02066302e-02 0.00000000e+00 0.00000000e+00
  8.07264447e-02 9.78117809e-02 2.29334123e-02 2.38395780e-02
  1.03342935e-01 0.00000000e+00 4.24865782e-02 0.00000000e+00
  5.61717674e-02 2.80754436e-02 9.73819196e-02 0.00000000e+00